In [1]:
import logging
import json


from pymongo import MongoClient
from importlib import resources
import os
import re
import datetime
import glob

from codex.database.vasp import generate_json as vasp_generate_json
from codex.database.espresso import run_helpdoc
from codex.database.vasp import generate_json as espresso_generate_json
from codex.database.utils import get_database

logging.basicConfig(level=logging.WARNING)

In [2]:
client = MongoClient('localhost', 27017)
#get_database(client, 'VASP', 'latest')

In [ ]:
nml = f90nml.read('example/espresso/si_bands.in')
query = []
for namelist, tags in nml.items():
    for tag in tags:
        query.append({'name': tag, 'section': namelist})
matches = []
for file_type in client['espresso-7^2'].list_collection_names():
    results = client['espresso-7^2'][file_type].find({'$or': query})
    if len(list(results)) == len(query):
        matches.append(file_type)

In [3]:
from codex.models.vasp_codex import VaspCodex

codex = VaspCodex('example/vasp/INCAR_Fe3O4.vasp', 'latest', client)

In [6]:
db = client['espresso-7^2']['pw']
print(db.find_one({'name': 'ecutwfdsjklc'}))


None


In [6]:
from pymatgen.io.vasp.inputs import Incar
incar = Incar.from_file('example/vasp/INCAR_Fe3O4.vasp')
incar2 = {'': incar}

# Add to MongoDB

In [ ]:
base_db_dir = resources.files("codex.database")

json_files = glob.glob(os.path.join(base_db_dir, "json", "*.json"))
for file in json_files:
    insert_json(client, file)

In [ ]:
database_names = client.list_database_names()
available_versions = [
    match.group(1)
    for f in database_names
    for match in [re.match(rf"{code}-([0-9]+\^[0-9]+(?:\^[0-9]+)?)", f)]
    if match
]
print(available_versions)

In [ ]:
get_database(client, 'Quantum ESPRESSO', '7.1')

# VASP database

In [ ]:
db = vasp_generate_json()

# QE database

In [ ]:
versions = ['6.7', '6.8', '7.0', '7.1', '7.2']
database_dir = resources.files("codex.database")

for v in versions:
    #run_helpdoc(v)
    db = espresso_generate_json(v)

# Generate Codex

In [ ]:
from importlib import resources
filename = 'example/sr3pbo_bands.in'
database_dir = resources.files("codex.database")
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist
codex = Codex(filename, version)


In [ ]:
print(codex.filename)
print(codex.code)
print(codex.package)
print(codex.tags['control'])

In [ ]:
from importlib import resources
import os
from base64 import b64decode
import json
import shutil

dbversion='7.2'
dir='static'

base_db_dir = resources.files("codex.database")
code = "qe"  # Need better terminology
database_dir = os.path.join(base_db_dir, f"{code}-{dbversion}")
database_filename = os.path.join(database_dir, "database.json")

with open(database_filename) as f:
    database = json.load(f)

working_dir = os.path.abspath(dir)
tags_dir = os.path.join(working_dir, "tags-qe")
if os.path.exists(tags_dir):
    shutil.rmtree(tags_dir, ignore_errors=True)
os.mkdir(tags_dir)
for p, package in database.items():
    print(p)
    os.mkdir(os.path.join(tags_dir, p))
    for nl, namelist in package.items():
        print(nl)
        os.mkdir(os.path.join(tags_dir, p, nl))
        for tag, v in namelist.items():
            print(tag, v)
            if v["html"] != 'No documentation was found for this tag.':
                webpage = b64decode(v["html"]).decode("utf-8")
                path = os.path.join(tags_dir, p, nl, f"{tag}.html")
                with open(path, "w") as f:
                    f.write(webpage)